In [2]:
import time
import math
import random
import os
import pathlib
import IPython.display as display
from PIL import Image
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2

from sklearn.metrics import confusion_matrix
from datetime import timedelta

%matplotlib inline

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
tf.__version__

'1.15.0'

In [0]:
#tf.enable_eager_execution()
#tf.compat.v1.disable_eager_execution()

In [0]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [0]:
CLASS_NAMES = ['cats','dogs']
NUM_CLASS = len(CLASS_NAMES)
CLASS_NAMES_TF = tf.convert_to_tensor(CLASS_NAMES, dtype=tf.string)
IMG_WIDTH,IMG_HEIGHT = 128,128

# Load Data

In [0]:
# data dir
#data_dir = os.path.join(os.getcwd(),'cnn_data/training_set')

In [0]:
data_dir = os.path.join(os.getcwd(),'/content/drive/My Drive/dataset/training_set')

In [0]:
# using pathlib
data_dir = pathlib.Path(data_dir)

In [11]:
data_dir

PosixPath('/content/drive/My Drive/dataset/training_set')

In [12]:
image_count = len(list(data_dir.glob('*/*.jpg')))
image_count

8000

In [13]:
for item in data_dir.glob('*'):
    print(item.name)

cats
dogs
.DS_Store


In [0]:
cats = list(data_dir.glob('cats/*'))

In [0]:
for image_path in cats[:3]:
    display.display(Image.open(str(image_path)))

In [15]:
data_dir

PosixPath('/content/drive/My Drive/dataset/training_set')

In [0]:
list_ds = tf.data.Dataset.list_files(str(data_dir/'*/*.jpg'))

In [0]:
val_dataset = list_ds.take(1000) 
train_dataset = list_ds.skip(1000)

In [0]:
len(list(train_dataset))

In [0]:
len(list(val_dataset))

In [0]:
#tf.shape(tf.get_default_graph().get_tensor_by_name('MatchingFiles:0'))[0]

In [0]:
def get_label(file_path):
    # Convert the path to a list of path components
    parts = tf.strings.split([file_path],sep=os.path.sep)
    
    label = tf.equal(CLASS_NAMES_TF,parts.values[-2])
    return label

In [0]:
def decode_img(img):
    # convert the compressed string to a 3D unit tensor
    img = tf.image.decode_jpeg(img,channels=3)
    # Use 'convert_image_dtype' to convert to floats in the [0,1] range.
    img  = tf.image.convert_image_dtype(img,tf.float32)
    # resize the image to the desired size.
    return tf.image.resize(img,[IMG_WIDTH,IMG_HEIGHT])

In [0]:
def process_path(file_path):
    label = get_label(file_path)
    # load the raw data from the file as a string
    img = tf.io.read_file(file_path)
    img = decode_img(img)
    return img,label

In [0]:
labeled_ds = list_ds.map(process_path,num_parallel_calls=AUTOTUNE)

In [0]:
val_dataset = val_dataset.map(process_path,num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.map(process_path,num_parallel_calls=AUTOTUNE)

In [0]:
#len(list(val_dataset))

In [0]:

val_dataset = val_dataset.batch(32).repeat()
train_dataset = train_dataset.batch(32).repeat()

val_osi = tf.compat.v1.data.make_one_shot_iterator(val_dataset)
# extract an element
val_dataset_next = val_osi.get_next()

train_osi = tf.compat.v1.data.make_one_shot_iterator(train_dataset)
# extract an element
train_dataset_next = train_osi.get_next()

In [27]:
train_dataset_next[0].shape

TensorShape([Dimension(None), Dimension(128), Dimension(128), Dimension(3)])

In [28]:
val_dataset

<DatasetV1Adapter shapes: ((?, 128, 128, 3), (?, 2)), types: (tf.float32, tf.bool)>

In [0]:
len(list(val_dataset ))

In [0]:
for image, label in train_dataset.take(1):
    print("Image shape: ", image.numpy().shape)
    print("Label: ", label.numpy())

In [0]:
dx = labeled_ds.batch(32).repeat()

In [0]:
osi = tf.compat.v1.data.make_one_shot_iterator(dx)
# extract an element
osi_next_element = osi.get_next()

In [0]:
iterator = dx.make_initializable_iterator()

In [0]:
next_element = iterator.get_next()

In [0]:
with tf.Session() as sess:
    sess.run(iterator.initializer)
    for i in range(100):
        val = sess.run(next_element)
        print(i)
        print(val[0].shape)

In [0]:
with tf.Session() as sess:  
    #sess.run(osi.initializer)
    for i in range(400):
        print(i)
        val = sess.run(osi_next_element)
        print(val[0].shape)

# Data Batches , shuffleing 

In [0]:
def prepare_for_training(ds,cache=True,shuffle_buffer_size=1000):
    
    # This is a small dataset, only load it once and keep it in memeory.
    # use '.cache(filename)' to chache preprocessing work for datasets that don't
    # fit in memory
    
    if cache:
        if isinstance(cache,str):
            ds = ds.cache(cache)
        else:
            ds = ds.cache()
    ds = ds.shuffle(buffer_size=shuffle_buffer_size)
    
    # Repeat forever
    
    ds = ds.repeat()
    
    ds = ds.batch(BATCH_SIZE)
    
    # 'prefetch' lets the dataset fetch batches in background while the model
    # is training.
    
    ds = ds.prefetch(buffer_size=AUTOTUNE)
    
    return ds

In [0]:
#train_ds = prepare_for_training(labeled_ds,cache=False)

#train_dataset = prepare_for_training(train_dataset,cache=False)
#val_dataset = prepare_for_training(val_dataset,cache=False)

#image_batch, label_batch = next(iter(train_dataset))

In [0]:
train_dataset

<DatasetV1Adapter shapes: ((?, 128, 128, 3), (?, 2)), types: (tf.float32, tf.bool)>

In [0]:
def show_batch(image_batch, label_batch):
    plt.figure(figsize=(10,10))
    for n in range(25):
        ax = plt.subplot(5,5,n+1)
        plt.imshow(image_batch[n])
        plt.title(CLASS_NAMES_TF[label_batch[n]][0].numpy().decode().title())
        plt.axis('off')

#show_batch(image_batch.numpy(), label_batch.numpy())

### Helper-function for plotting images

In [0]:
def plot_imgs(imgs,cls_true,cls_pred=None):
    
    if len(imgs) == 0:
        print('No images to show')
    else:
        random_indeces = random.sample(range(len(imgs)),min(len(imgs),9))
        
    if cls_pred is not None:
        
        imgs,cls_true,cls_pred = zip(*[(imgs[i],cls_true[i],cls_pred[i]) for i in random_indeces])
                                 
    else:
        imgs,cls_true = zip(*[(imgs[i],cls_true[i]) for i in random_indeces])
        
    
    # Creationg figure with 3X3 subplots
    fig,axes = plt.subplots(3,3)
    fig.subplots_adjust(hspace=0.3,wspace = 0.3)
    
    for i,ax in enumerate(axes.flat):
        
        ax.imshow(imgs[i])
        
        # show true and predicted classes
        
        if cls_pred is None:
            xlabel = 'True:{0}'.format(CLASS_NAMES[np.argmax(cls_true[i])])
        else:
            xlabel = 'True:{0}, Pred:{1}'.format(cls_true[i],cls_pred[i])
            
        # Show the classes as the label on the x axis   
        ax.set_xlabel(xlabel)
        
        # remove ticks from the plot
        ax.set_xticks([])
        ax.set_yticks([])
        
    # Ensure the plot is shown correctly with multiple plots
    # in a single Notebook cell.
    plt.show()
    

In [0]:
#plot_imgs(image_batch.numpy(),label_batch)

# Configuration and Hyperparameters

In [0]:
## Convolutional Layer 1
filter_size1 = 3
num_filters1 = 32

# Convolutional Layer 2
filter_size2 = 3
num_filters2 = 32

#Convolutional Layer 3
filter_size3 = 3
num_filters3 = 64

#Fully-connected layer
fc_size = 128    # Number of neurons in fully-connected layer.

#Number of color channales for the images: 
num_channels = 3

# image dimensions
img_width = 128
img_hight =128
img_size = 128


# Size of image when flattened to a single dimension
img_size_flat = img_size * img_size * num_channels

# Tuple with height and width of images used to reshape arrays.
img_shape = (img_size,img_size)

#class info
classes = ['dogs','cats']
num_classes = len(classes)

# batch size
batch_size = 32

#validation split
validation_size = 0.16

# How long to wait after valication loss stops imporving before terminating
early_stopping = None # use None if you don't want to implement early stoping



### Helper-functions for creating new variables

In [0]:
def new_weights(shape):
    return tf.Variable(tf.truncated_normal(stddev=0.05,shape=shape))

def new_baises(length):
    return tf.Variable(tf.constant(0.05, shape=[length]))

### Helper-function for creating a new Convolutional Layer

In [0]:
def new_conv_layer(input,               # The previous layer
                   num_input_channels,  # Num of channels in previous layer
                   filter_size,         # Width and hight of each filter
                   num_filters,          # Num of filters
                   use_pooling=True,    # Use 2X2 max pooling
                  ):
    
    # Shape of filter weight for convolution
    # This format is determind by the tensorflow
    shape = [img_size,img_size,num_input_channels,num_filters]
    
    # Create new weights aka. filters with the given shape.
    weights = new_weights(shape=shape)
    
    
    # Create new biases, one for each filter.
    biases = new_baises(length=num_filters)
    
    #tf convolution layer
    layer = tf.nn.conv2d(input = input,   
                         filters = weights,
                         strides =[1,1,1,1],
                         padding = 'SAME',     
                         )
    
    layer += biases
    
    if use_pooling:
        
        layer = tf.nn.max_pool(value=layer,
                               ksize=[1, 2, 2, 1],
                               strides=[1, 2, 2, 1],
                               padding='SAME')
    
    # Rectified Linear Unit (ReLU).
    layer = tf.nn.relu(layer)
    
    # We return both the resulting layer and the filter-weights
    # because we will plot the weights later.
    return layer, weights


### Helper-function for flattening a layer

In [0]:
def flatten_layer(layer):
    
    #get the shape of the input layer
    layer_shape = layer.get_shape()
    
    # The shape of the input layer is assumed to be:
    # layer_shape == [num_images, img_height, img_width, num_channels]

    # The number of features is: img_height * img_width * num_channels
    # We can use a function from TensorFlow to calculate this.
    num_features = layer_shape[1:4].num_elements()
    
    #If one component of shape is the special value -1, 
    #the size of that dimension is computed so that the total size remains constant. 
    #In particular, a shape of [-1] flattens into 1-D. At most one component of shape can be -1.
    
    # Reshape the layer to [num_images, num_features].
    # Note that we just set the size of the second dimension
    # to num_features and the size of the first dimension to -1
    # which means the size in that dimension is calculated
    # so the total size of the tensor is unchanged from the reshaping.
    
    layer_flat = tf.reshape(layer, [-1, num_features])
    
    # The shape of the flattened layer is now:
    # [num_images, img_height * img_width * num_channels]

    # Return both the flattened layer and the number of features.
    return layer_flat, num_features
    
    

### Helper-function for creating a new Fully-Connected Layer

In [0]:
def new_fc_layer(input,input_size,output_size,use_relu=True):
    
    shape = [input_size,output_size]
    
    weights = new_weights(shape=shape)
    
    biases = new_baises(output_size)
    
    # Calculate the layer as the matrix multiplication of
    # the input and weights, and then add the bias-values.
    layer = tf.matmul(input, weights) + biases

    # Use ReLU?
    if use_relu:
        layer = tf.nn.relu(layer)

    return layer


### Placeholder variables

In [0]:
#tf.compat.v1.disable_eager_execution()

In [0]:
x = tf.placeholder(dtype=tf.float32,shape=[None,img_size_flat],name='x')

In [41]:
x

<tf.Tensor 'x:0' shape=(?, 49152) dtype=float32>

In [0]:
x_image = tf.reshape(x,[-1, img_size, img_size, num_channels])

In [43]:
x_image

<tf.Tensor 'Reshape:0' shape=(?, 128, 128, 3) dtype=float32>

In [0]:
y_true = tf.placeholder(dtype=tf.float32,shape=[None,num_classes],name='y_true')

In [0]:
y_true_cls = tf.argmax(y_true,axis=1)

### Convolutional Layer 1

In [46]:
x_image

<tf.Tensor 'Reshape:0' shape=(?, 128, 128, 3) dtype=float32>

In [0]:
layer_conv1,weights_conv1 = new_conv_layer(input= x_image,
                                            num_input_channels=num_channels,
                                            filter_size=filter_size1,
                                            num_filters=num_filters1)

In [48]:
layer_conv1,weights_conv1

(<tf.Tensor 'Relu:0' shape=(?, 64, 64, 32) dtype=float32>,
 <tf.Variable 'Variable:0' shape=(128, 128, 3, 32) dtype=float32_ref>)

### Convolutional Layers 2 and 3

In [0]:
layer_conv2, weights_conv2 = \
    new_conv_layer(input=layer_conv1,
                   num_input_channels=num_filters1,
                   filter_size=filter_size2,
                   num_filters=num_filters2,
                   use_pooling=True)

In [50]:
layer_conv2

<tf.Tensor 'Relu_1:0' shape=(?, 32, 32, 32) dtype=float32>

In [0]:
layer_conv3, weights_conv3 = \
    new_conv_layer(input=layer_conv2,
                   num_input_channels=num_filters2,
                   filter_size=filter_size3,
                   num_filters=num_filters3,
                   use_pooling=True)

In [52]:
layer_conv3

<tf.Tensor 'Relu_2:0' shape=(?, 16, 16, 64) dtype=float32>

### Flatten Layer

In [0]:
layer_flat, num_features = flatten_layer(layer_conv3)

In [54]:
layer_flat, num_features

(<tf.Tensor 'Reshape_1:0' shape=(?, 16384) dtype=float32>, 16384)

In [55]:
16*16*64

16384

### Fully-Connected Layer 1

In [0]:
layer_fc1 = new_fc_layer(input=layer_flat,
                         input_size=num_features,
                         output_size=fc_size,
                         use_relu=True)

In [57]:
layer_fc1

<tf.Tensor 'Relu_3:0' shape=(?, 128) dtype=float32>

### Fully-Connected Layer 2

In [0]:
layer_fc2 = new_fc_layer(input=layer_fc1,
                         input_size=fc_size,
                         output_size=32,
                         use_relu=False)

In [0]:
layer_fc2 = new_fc_layer(input=layer_fc1,
                         input_size=fc_size,
                         output_size=num_classes,
                         use_relu=False)

In [59]:
layer_fc2

<tf.Tensor 'add_4:0' shape=(?, 2) dtype=float32>

### Fully-Connected Layer 3

In [0]:
layer_fc3 = new_fc_layer(input=layer_fc2,
                         input_size=32,
                         output_size=2,
                         use_relu=False)

In [0]:
layer_fc3

### Predicted Class

In [0]:
y_pred = tf.nn.softmax(layer_fc2)

In [0]:
y_pred_cls = tf.argmax(y_pred, axis=1)

### Cost-function to be optimized

In [0]:
cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(logits=layer_fc2, labels=y_true)

In [0]:
cost = tf.reduce_mean(cross_entropy)

### Optimization Method

In [0]:
optimizer = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(cost)

### Performance Measures

In [0]:
correct_prediction = tf.equal(y_pred_cls, y_true_cls)

In [0]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

### Create TensorFlow session

In [0]:
session = tf.Session()

In [0]:
#Initialize variables
session = tf.Session()
session.run(tf.global_variables_initializer())

### Helper-function to perform optimization iterations

In [67]:
batch_size

32

In [68]:
num_batches = int(7000/batch_size)
num_batches

218

In [69]:
epochs = 100
epochs

100

In [0]:
def print_progress(epoch,feed_dict_train,feed_dict_validation,val_loss):
    
    #calculate the accuracy on the training set
    
    acc = session.run(accuracy, feed_dict=feed_dict_train)
    val_acc = session.run(accuracy,feed_dict=feed_dict_validation)
    
    msg = "Epoch {0} ---Training Accuracy: {1:>6.1%}, Validation Accuracy: {2:>6.1%}, Validation Loss: {3:.3f}"
    
    print(msg.format(epoch + 1, acc, val_acc, val_loss))
    
    
    

In [0]:
# Counter for total number of iterations performed so for

total_iterations = 0

def optimize(epochs):
    
    # Ensure we update the global variable rather than a local copy.
    #global total_iterations
    
    # Start-time used for printing time-usage below.
    start_time = time.time()
    
    best_val_loss = float('inf')
    patience = 0
    
    for epoch in range(epochs):
        
    
        for i in range(num_batches):

            print(i)

            #Get a batch of training examples.
            
            image_batch_train, label_batch_train =session.run(train_dataset_next)

            feed_dict_train = {x_image: image_batch_train,
                               y_true: label_batch_train}
            
            session.run(optimizer, feed_dict=feed_dict_train)

            #train_acc = session.run(fetches=[accuracy], feed_dict=feed_dict_train)

        # loss and accuracy for each epoch..

        image_batch_val, label_batch_val = session.run(val_dataset_next)

        feed_dict_validate = {x_image: image_batch_val,y_true: label_batch_val}

        val_loss = session.run(fetches=[cost], feed_dict=feed_dict_validate)
        train_acc = session.run(fetches=[accuracy], feed_dict=feed_dict_train)
        val_acc  = session.run(fetches=[accuracy], feed_dict=feed_dict_validate)
        
        print(f'Epoch:{epoch} | Train Acc:{train_acc} | Validation acc:{val_acc}| val_loss:{val_loss}')


In [0]:
#Initialize variables
session = tf.Session()
session.run(tf.global_variables_initializer())

optimize(10)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
Epoch:0 | Train Acc:[0.46875] | Validation loss:[0.625]| val_loss:[0.96584135]
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
5

In [0]:
session.close()

### Helper-function for plotting convolutional weights